Weaviate Demo based on https://weaviate.io/developers/weaviate/quickstart/local.

Prerequisites:
- Deploy Weaviate with k8s or docker.
- Ollama

## Warm Up Ollama

In [6]:
%%bash
#!/bin/bash
export OLLAMA_ENDPOINT=http://host.docker.internal:30491
curl ${OLLAMA_ENDPOINT}/api/generate -d '{"model": "llama3.2:3b","prompt": "Tell me a joke."}'
curl ${OLLAMA_ENDPOINT}/api/embed -d '{"model": "nomic-embed-text:latest","input": "Ollama makes it easy to run LLMs locally."}'
curl ${OLLAMA_ENDPOINT}/api/embeddings -d '{"model": "nomic-embed-text:latest","input": "Ollama makes it easy to run LLMs locally."}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   135  100    83  100    52    418    262 --:--:-- --:--:-- --:--:--   681


{"error":"model requires more system memory (3.4 GiB) than is available (2.8 GiB)"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89    0     0  100    89      0     88  0:00:01  0:00:01 --:--:--    89

{"model":"nomic-embed-text:latest","embeddings":[[0.044281255,0.047669165,-0.16548696,-0.05635137,0.0042649573,-0.066425264,0.009431725,-0.016140103,-0.021452457,-0.0009315589,0.020742895,0.062057912,0.056323867,-0.02631032,-0.028445452,0.018662414,-0.012527076,-0.03170685,-0.0322611,-0.058564946,-0.022173805,-0.0071653835,0.029808793,-0.06823393,0.07429899,0.019060558,-0.008764805,-0.0022234428,0.03232139,-0.015043626,0.08671801,-0.07593729,0.03245573,-0.06395427,-0.0040697856,-0.019920165,0.048566025,-0.06228461,-0.0095454585,0.039712455,0.03406245,-0.031747177,-0.005548129,-0.00591153,0.09064888,-0.0038484803,0.026048332,0.037925392,0.02609509,-0.03473219,-0.021886323,-0.07886914,0.0023872647,0.0053564226,0.030721718,0.019113088,-0.056394145,0.08635942,0.005537157,-0.056527298,0.05709834,-0.029258613,-0.04981896,0.052696474,-0.032145973,-0.006807124,-0.04789264,0.00012378574,-0.011545516,-0.0040812856,-0.0014343859,0.010890339,0.009207024,-0.038983043,-0.014262754,0.03255573,-0.0328

100  9715    0  9626  100    89   8882     82  0:00:01  0:00:01 --:--:--  8970
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   105  100    16  100    89   1513   8417 --:--:-- --:--:-- --:--:--     0- --:--:-- 10500


{"embedding":[]}

## Create Collection and Injest Data

In [8]:
import weaviate

client = weaviate.connect_to_custom(
    http_host="host.docker.internal", # inside devcontainer
    http_port=30389,
    http_secure=False,
    grpc_host="host.docker.internal", # inside devcontainer
    grpc_port=30925,
    grpc_secure=False
)
print(client.is_ready())  # Should print: `True`

True


In [9]:
from weaviate.classes.config import Configure
import json

data_file = "weaviate-dev/jeopardy_tiny.json"
ollama_endpoint = "http://myrelease-ollama:11434" # inside weaviate pod

def init_collection():
    client.collections.delete(name="Question")
    questions = client.collections.create(
        name="Question",
        vectorizer_config=Configure.Vectorizer.text2vec_ollama(     
            api_endpoint=ollama_endpoint,
            model="nomic-embed-text:latest",
        ),
        generative_config=Configure.Generative.ollama( 
            api_endpoint=ollama_endpoint,
            model="llama3.2:3b",
        )
    )

def injest_data():
    # add data to collection
    with open(data_file, "r") as ifile:
        data = json.load(ifile)
        questions = client.collections.get("Question")
        with questions.batch.fixed_size(batch_size=200) as batch:
            for d in data:
                batch.add_object(
                    {
                        "answer": d["Answer"],
                        "question": d["Question"],
                        "category": d["Category"],
                    }
                )
                if batch.number_errors > 2:
                    print("Batch import stopped due to excessive errors.")
                    break

        failed_objects = questions.batch.failed_objects
        if failed_objects:
            print(f"Number of failed imports: {len(failed_objects)}")
            print(f"First failed object: {failed_objects[0]}")

if __name__ == "__main__":
    try:
        init_collection()
        injest_data()
    except Exception as e:
        print(f"Error: {e}")

/home/admin/.pyenv/versions/3.11.11/lib/python3.11/site-packages/weaviate/collections/classes/config.py:1950: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


## Semantic Sesarch instead of keyword

In [10]:
import json

questions = client.collections.get("Question")

# semantic search instead of keywork search
response = questions.query.near_text(
    query="biology",
    limit=2
)
for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

{
  "answer": "DNA",
  "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance",
  "category": "SCIENCE"
}
{
  "answer": "Liver",
  "question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "category": "SCIENCE"
}


## RAG

In [11]:
questions = client.collections.get("Question")

# RAG
response = questions.generate.near_text(
    query="biology",
    limit=2,
    grouped_task="Write a tweet with emojis about these facts."
)
print(response.generated)

"Did you know? 🤯 DNA is the molecule that holds our genetic code! 💡 Meanwhile, in the human body, the liver is hard at work removing excess glucose and storing it as glycogen. Talk about a vital role! ❤️ #ScienceFacts #Genetics #LiverFacts"


In [12]:
client.close()